# GradCAM Evaluation : CNN

### GradCAM Implementation

In [4]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split

## Load data

In [5]:
# File paths
image_dir = '../images'
labels_df = pd.read_csv('../labels.csv')

# Extract file names and labels
image_filenames = labels_df['Filename'].values
y = labels_df['Label'].values

# Construct full image paths
image_paths = [os.path.join(image_dir, fname) for fname in image_filenames]

# Ensure images and labels are aligned
if len(image_paths) != len(y):
    raise ValueError("Number of images and labels do not match.")

labels_df.head()

,Filename,Label,Species
0,20160928-140314-0.jpg,0,Chinee apple
1,20160928-140337-0.jpg,0,Chinee apple
2,20160928-140731-0.jpg,0,Chinee apple
3,20160928-140747-0.jpg,0,Chinee apple
4,20160928-141107-0.jpg,0,Chinee apple


# Train test split

In [6]:
# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(image_paths, y, test_size=0.2, stratify=y, random_state=42)

# Create label-to-species mapping for displaying label counts
label_to_species = dict(zip(labels_df['Label'], labels_df['Species']))

# Label distribution
def print_label_distribution(y_train, y_test, label_to_species):
    # Create label counts for train and test datasets
    train_label_counts = pd.Series(y_train).value_counts().reset_index()
    test_label_counts = pd.Series(y_test).value_counts().reset_index()

    # Rename columns
    train_label_counts.columns = ['Label', 'Count']
    test_label_counts.columns = ['Label', 'Count']

    # Map species to the label counts
    train_label_counts['Species'] = train_label_counts['Label'].map(label_to_species)
    test_label_counts['Species'] = test_label_counts['Label'].map(label_to_species)

    # Display the distributions
    print("Training dataset distribution:")
    print(train_label_counts.to_string(index=False))

    print("\nTesting dataset distribution:")
    print(test_label_counts.to_string(index=False))

print_label_distribution(y_train, y_test, label_to_species)

Training dataset distribution:
 Label  Count        Species
     8   7285       Negative
     0    900   Chinee apple
     6    859      Siam weed
     1    851        Lantana
     4    849 Prickly acacia
     2    825    Parkinsonia
     3    818     Parthenium
     7    813     Snake weed
     5    807    Rubber vine

Testing dataset distribution:
 Label  Count        Species
     8   1821       Negative
     0    225   Chinee apple
     6    215      Siam weed
     1    213        Lantana
     4    213 Prickly acacia
     2    206    Parkinsonia
     3    204     Parthenium
     7    203     Snake weed
     5    202    Rubber vine


## Dealing with imbalance data

To mitigate class imbalance, I will randomly select Negative instances (Label is 8) to balance with other classes. 
Undersampling will only be performed on the training set to prevent the model from overfitting to the Negative instances. 

In [7]:
# Function to perform undersampling
def undersample_classes(X_train, y_train, target_size=800):
    # Create DataFrame with image paths and corresponding labels
    train_df = pd.DataFrame({'Filename': X_train, 'Label': y_train})
    
    # Initialize empty list to hold undersampled data
    undersampled_df = []

    # Iterate through each class (label)
    for label in np.unique(y_train):
        # Get the subset of data for this class
        class_subset = train_df[train_df['Label'] == label]
        
        # Sample 'target_size' number of images from each class
        class_sample = class_subset.sample(n=target_size, random_state=42)
        
        # Append the sampled data to the list
        undersampled_df.append(class_sample)
    
    # Combine the undersampled data into one DataFrame
    undersampled_df = pd.concat(undersampled_df, ignore_index=True)
    
    # Return the undersampled image paths and labels
    X_train_balanced = undersampled_df['Filename'].values
    y_train_balanced = undersampled_df['Label'].values
    
    return X_train_balanced, y_train_balanced

# Perform the undersampling
X_train_balanced, y_train_balanced = undersample_classes(X_train, y_train, target_size=800)


In [8]:
# Split balanced training dataset into training and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_balanced, y_train_balanced, test_size=0.125, stratify=y_train_balanced, random_state=42)

## 1. Pre-processing (resizing, normalization, edge detection)

In [9]:
import tensorflow as tf
import numpy as np
import cv2  # For edge detection

# Function to load and decode images
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)  # Decode as RGB image
    return image

# Preprocess image: resize and normalize
def preprocess_image(image, target_size=(224, 224)):
    image = tf.image.resize(image, target_size)  # Resize image to the target size
    image = image / 255.0  # Normalize to [0, 1]
    return image

# Edge detection using TensorFlow's sobel_edges
def edge_detection_tf(image):
    gray = tf.image.rgb_to_grayscale(image)  # Convert RGB to grayscale
    
    # Add a batch dimension for the edge detection function
    gray = tf.expand_dims(gray, axis=0)  # Add batch dimension (shape: [1, H, W, 1])
    
    sobel_edges = tf.image.sobel_edges(gray)  # Returns gradients in x and y
    
    # Remove batch dimension after sobel operation (result shape: [H, W, 2])
    sobel_edges = tf.squeeze(sobel_edges, axis=0)
    
    # Compute edge magnitude (magnitude of gradients)
    edge_magnitude = tf.sqrt(tf.reduce_sum(tf.square(sobel_edges), axis=-1))  # Compute edge magnitude
    
    # Resize edge magnitude to match the image size
    edge_magnitude = tf.image.resize(edge_magnitude, [224, 224])  # Resize if needed
    return edge_magnitude


# Function to combine original image and edge detection result
def combine_image_and_edges(image):
    edges = edge_detection_tf(image)  # Perform edge detection using TensorFlow
    image_with_edges = tf.concat([image, edges], axis=-1)  # Combine RGB image with edge features
    return image_with_edges

# Create the dataset for training and testing
def create_dataset(image_paths, labels, batch_size=32, augment=False):
    X_processed = []
    y_processed = []

    for image_path, label in zip(image_paths, labels):
        # Load and preprocess the image
        image = load_image(image_path)
        image = preprocess_image(image)
        
        # Combine the RGB image with edges
        image_with_edges = combine_image_and_edges(image)
        
        # Append processed image and label
        X_processed.append(image_with_edges)
        y_processed.append(label)

    # Convert lists to NumPy arrays for further use
    X_processed = np.array(X_processed)
    y_processed = np.array(y_processed)

    return X_processed, y_processed

X_train_processed, y_train_processed = create_dataset(X_train, y_train)
X_val_processed, y_val_processed = create_dataset(X_val, y_val)
X_test_processed, y_test_processed = create_dataset(X_test, y_test)


In [10]:
from tensorflow.keras.utils import to_categorical

# Perform one-hot encoding for the labels
y_train_processed = to_categorical(y_train_processed, num_classes=9)
y_val_processed = to_categorical(y_val_processed, num_classes=9)
y_test_processed = to_categorical(y_test_processed, num_classes=9)

## 2. Train CNN model

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, Input
from tensorflow.keras.optimizers import Adam

In [12]:
# Define the model
inputs = Input(shape=(224, 224, 4)) 

x = layers.Conv2D(64, (3, 3), padding="same", strides=2)(inputs)
x = layers.Conv2D(64, (3, 3), padding="same", strides=2)(x)
x = layers.BatchNormalization()(x)
layer1 = layers.ReLU()(x)

x = layers.Conv2D(64, (3, 3), padding="same")(layer1)
x = layers.Conv2D(64, (3, 3), padding="same")(x)
x = layers.BatchNormalization()(x)
layer2 = layers.ReLU()(x)

x = layers.Conv2D(128, (3, 3), padding="same")(layer2)
x = layers.Conv2D(128, (3, 3), padding="same")(x)
x = layers.BatchNormalization()(x)
layer3 = layers.ReLU()(x)

x = layers.Conv2D(128, (3, 3), padding="same")(layer3)
x = layers.Conv2D(128, (3, 3), padding="same")(x)
x = layers.BatchNormalization()(x)
layer4 = layers.ReLU()(x)

x = layers.Conv2D(256, (3, 3), padding="same", strides=2)(layer4)
x = layers.Conv2D(256, (3, 3), padding="same", strides=2)(x)
x = layers.BatchNormalization()(x)
layer5 = layers.ReLU()(x)

x = layers.GlobalMaxPooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(9, activation="softmax")(x)

In [ ]:
model = tf.keras.Model(inputs, outputs, name="cnn_model")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy", "Precision", "Recall"])

history = model.fit(X_train_processed, 
                    y_train_processed, 
                    epochs=20, 
                    batch_size=32, 
                    validation_data=(X_val_processed, y_val_processed))

Epoch 1/20
197/197 ━━━━━━━━━━━━━━━━━━━━ 483s 2s/step - Precision: 0.2030 - Recall: 0.1131 - accuracy: 0.1955 - loss: 3.1984 - val_Precision: 0.3519 - val_Recall: 0.0211 - val_accuracy: 0.1189 - val_loss: 3.1622
Epoch 2/20
 70/197 ━━━━━━━━━━━━━━━━━━━━ 5:06 2s/step - Precision: 0.4206 - Recall: 0.1006 - accuracy: 0.3065 - loss: 1.8018

In [ ]:
y_pred_cnn = model.predict(X_test_processed)

y_pred_classes = np.argmax(y_pred_cnn, axis=1)
y_test_processed = np.argmax(y_test_processed, axis=1)

110/110 [==============================] - 136s 1s/step


In [3]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report

# GradCAM class remains the same as provided earlier

class GradCAM:
    def __init__(self, model, classIdx, layerName=None):
        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName

        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):
        for layer in reversed(self.model.layers):
            if isinstance(layer, tf.keras.layers.Conv2D):
                return layer.name
        raise ValueError("Could not find a Conv2D layer. Cannot apply GradCAM.")

    def compute_heatmap(self, image, eps=1e-8):
        gradModel = tf.keras.models.Model(
            inputs=[self.model.inputs],
            outputs=[self.model.get_layer(self.layerName).output, self.model.output]
        )
        with tf.GradientTape() as tape:
            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]

        grads = tape.gradient(loss, convOutputs)
        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads

        weights = tf.reduce_mean(guidedGrads, axis=(0, 1, 2))
        convOutputs = convOutputs[0]
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)

        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))

        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")
        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5, colormap=cv2.COLORMAP_JET):
        heatmap = cv2.applyColorMap(heatmap, colormap)
        if image.shape[-1] == 4:
            image = image[:, :, :3]
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)
        return (heatmap, output)

# Function for GradCAM visualization
def gradcam_visualize(model, X_test, y_test, num_classes=9, samples=5):
    for i in range(samples):
        test_image = X_test[i]
        true_label = np.argmax(y_test[i])
        pred_probs = model.predict(np.expand_dims(test_image, axis=0))
        pred_label = np.argmax(pred_probs)

        cam = GradCAM(model, pred_label)
        heatmap = cam.compute_heatmap(np.expand_dims(test_image, axis=0))
        (heatmap, overlay) = cam.overlay_heatmap(heatmap, test_image, alpha=0.6)

        plt.figure(figsize=(15, 10))
        plt.subplot(1, 2, 1)
        plt.imshow(test_image[:, :, :3])
        plt.title(f"True: {true_label}, Predicted: {pred_label}")
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
        plt.title("GradCAM Overlay")
        plt.axis("off")
        plt.show()

# Evaluate CNN model
def evaluate_model_with_gradcam(model, X_test, y_test, num_classes=9):
    # Predict labels
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Classification report
    print("Classification Report:")
    print(classification_report(y_true, y_pred))

    # Confusion matrix
    confusion_matrix = np.zeros((num_classes, num_classes))
    for i in range(len(y_pred)):
        confusion_matrix[y_true[i], y_pred[i]] += 1

    confusion_matrix = confusion_matrix / confusion_matrix.sum(axis=1, keepdims=True) * 100

    plt.figure(figsize=(10, 8))
    sns.heatmap(confusion_matrix, annot=True, fmt=".2f", cmap="viridis", xticklabels=range(num_classes), yticklabels=range(num_classes))
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    plt.title("Confusion Matrix (%)")
    plt.show()

    # GradCAM visualization
    gradcam_visualize(model, X_test, y_test, num_classes=num_classes, samples=5)

# Example Usage
# Assume `X_test_processed` and `y_test_processed` are preprocessed test data and labels
evaluate_model_with_gradcam(model, X_test_processed, y_test_processed)


NameError: name 'X_test_processed' is not defined